# Vasanth Mani Vasi

In [ ]:
from google.colab import files
files.upload()

In [3]:
!unzip archive.zip

Archive:  archive.zip
  inflating: spam.csv                


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [7]:
df = pd.read_csv('/content/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [9]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [10]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [11]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [12]:
inp = Input(name='inputs',shape=[max_len])
x = Embedding(max_words,50,input_length=max_len)(inp)
x = LSTM(64)(x)
x = Dense(256,name='FC1')(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(1,name='out_layer')(x)
x = Activation('sigmoid')(x)
model = Model(inputs=inp,outputs=x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [13]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)
model.save('sms_classifier.h5')

Epoch 1/10
30/30 [==============================] - 12s 270ms/step - loss: 0.3450 - accuracy: 0.8662 - val_loss: 0.1506 - val_accuracy: 0.9451
Epoch 2/10
30/30 [==============================] - 7s 249ms/step - loss: 0.0949 - accuracy: 0.9781 - val_loss: 0.0534 - val_accuracy: 0.9895
Epoch 3/10
30/30 [==============================] - 8s 272ms/step - loss: 0.0468 - accuracy: 0.9871 - val_loss: 0.0514 - val_accuracy: 0.9895
Epoch 4/10
30/30 [==============================] - 10s 323ms/step - loss: 0.0327 - accuracy: 0.9913 - val_loss: 0.0531 - val_accuracy: 0.9895
Epoch 5/10
30/30 [==============================] - 9s 295ms/step - loss: 0.0237 - accuracy: 0.9931 - val_loss: 0.0742 - val_accuracy: 0.9863
Epoch 6/10
30/30 [==============================] - 7s 247ms/step - loss: 0.0214 - accuracy: 0.9942 - val_loss: 0.0618 - val_accuracy: 0.9873
Epoch 7/10
30/30 [==============================] - 7s 249ms/step - loss: 0.0145 - accuracy: 0.9960 - val_loss: 0.0934 - val_accuracy: 0.9852
Epoc

In [15]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test Loss: {:0.3f}\nTest Accuracy: {:0.2f}%'.format(accr[0],accr[1]*100))

27/27 [==============================] - 1s 21ms/step - loss: 0.0707 - accuracy: 0.9833
Test Loss: 0.071
Test Accuracy: 98.33%
